In [1]:
import numpy as np
import pandas as pd
import pickle
import json
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
def str_to_dt(datestr):
    return dt.datetime.strptime(datestr, '%Y/%m/%d').date()
def dt_to_str(datedt):
    return datedt.strftime('%Y/%m/%d')

In [3]:
def save_obj(obj, name ):
    with open('../objs/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('../objs/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

Four necessary dictionaries are loaded:
- ID_city: mapping of ID (`int`) to name of city (or metapopulation, which means a country in this code, `str`)
- city_ID: mapping of name of city (or metapopulation, which means a country in this code, `str`) to ID (`int`)
- ID_net: `ID_net[(i, j)]` indicates number of transition between metapopulation i to j, `int`
- city_net: `ID_net[(cityname_i, cityname_j)]` indicates number of transition between metapopulation i to j, `int`

In [4]:
ID_city = load_obj('ID_city_Globalcountrytop')
city_ID = load_obj('city_ID_Globalcountrytop')
ID_net = load_obj('ID_net_Globalcountrytop')
city_net = load_obj('city_net_Globalcountrytop')

In [5]:
len(ID_net)

2854

Epidemic data of each metapopulation loaded:

- `ID`, `Country` column should have same setting as the four dicts above, denoting each metapopulation
- `Population` is used as initial susceptible of each metapopulation
- the remaining columns are cumulative confirmed cases of each metapopulation for plotting graph 

In [6]:
origin_data = pd.read_csv('../data/global_epi_0210_0307.csv')

In [7]:
origin_data.head()

,ID,Country,Population,2020/2/10,2020/2/11,2020/2/12,2020/2/13,2020/2/14,2020/2/15,2020/2/16,...,2020/2/27,2020/2/28,2020/2/29,2020/3/1,2020/3/2,2020/3/3,2020/3/4,2020/3/5,2020/3/6,2020/3/7
0,0,South Korea,51225308,27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7041
1,1,Italy,60550075,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5883
2,2,Iran,82913906,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5823
3,3,France,65129728,11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,949
4,4,Germany,83517045,14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,799


In [8]:
alldates = list(origin_data.columns)[3:]

In [9]:
population = origin_data[['ID', 'Country', 'Population']]

In [10]:
epidemic = origin_data.drop(['Population',], axis= 1)

In [11]:
epidemic.head()

,ID,Country,2020/2/10,2020/2/11,2020/2/12,2020/2/13,2020/2/14,2020/2/15,2020/2/16,2020/2/17,...,2020/2/27,2020/2/28,2020/2/29,2020/3/1,2020/3/2,2020/3/3,2020/3/4,2020/3/5,2020/3/6,2020/3/7
0,0,South Korea,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7041
1,1,Italy,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5883
2,2,Iran,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5823
3,3,France,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,949
4,4,Germany,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,799


In [12]:
epidemic.shape

(100, 29)

In [13]:
epidemic = np.array(epidemic)

In [14]:
'''
N: total population of each region, order by ID
initI: first day's infection num, order by ID
'''
firstday = '2020/2/10'
N = []
initI = []
for i, row in origin_data.iterrows():
    N.append(float(row['Population']))
    initI.append(float(row[firstday]))

In [15]:
population_size = len(N)

In [16]:
ID_net_full = {}
for i in range(population_size):
    for j in range(population_size):
        if i == j:
            continue
        try:
            ID_net_full[(i, j)] = ID_net[(i, j)] / 20.
        except:
            ID_net_full[(i, j)] = 0.

In [17]:
def sir_full(net,N,d,a,r,q1,q2): # full breatout prediction
    l = epidemic.shape[0] # number of metapopulations
    S = np.zeros((l, d)) # d day's susceptible num
    I = np.zeros((l, d)) # d day's infectious num
    R = np.zeros((l, d)) # d day's recover/death num
    deltaI = np.zeros((d,))
    S[:, 0] = N

    I[:, 0] = np.array(initI) # first day's infectious num
    deltaI[0] = np.sum(initI)
    
    pops = []

    for i in range(d-1):
        for j in range(l):
            M = 0
            for k in range(l):
                try:
                    if net[j, k] and I[k, i] and N[k]:
                        M += (net[j, k])*I[k, i]/N[k]
                except:
                    pass
            M = M if M>0 else 0
            if N[j] != 0:
                dI = a*S[j,i]*(q1*I[j,i] + q2*M)/N[j] # a is alpha, q1, q2 is parameters balance weights between local and exchange metapopulation
            else:
                dI = a*S[j,i]*(q1*I[j,i] + q2*M)
            S[j,i+1] = S[j,i] - dI
            I[j,i+1] = I[j,i] + dI - r*I[j,i]
            R[j,i+1] = R[j,i] + r*I[j, i] # r is beta
            if(S[j,i+1] <0):
                S[j,i+1] = 0
            if(I[j,i+1]<0):
                I[j,i+1] = 0
            
            deltaI[i+1] += dI
            if deltaI[i+1] < 0:
                deltaI[i+1] = 0
    return (S,I,R), deltaI, pops

In [18]:
# These parameters can be hand-crafted or obtained using some optimization algorithm like MCMC
a = 0.075275 # a is alpha, q1, q2 is parameters balance weights between local and exchange metapopulation
r = 0.001925 # r is beta
q1 = 1.
q2 = 0.00002 
NDays = 120
(S,I,R), deltaI, pops = sir_full(ID_net_full,N,NDays,a,r,q1,q2)

In [19]:
alldates_dt = [str_to_dt(x) for x in alldates]
t_range_dt = [alldates_dt[0] + dt.timedelta(days=x) for x in range(NDays)]

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(alldates_dt, np.sum(epidemic[:, 2:], axis = 0), 'k+-', label = 'truth')
plt.plot(t_range_dt, np.sum(I, axis = 0), 'b--', label = 'I')
plt.plot(t_range_dt, np.sum(I + R, axis = 0), 'r--', label = 'I + R')
plt.title(u'Global Prediction (Start from {})'.format(alldates[0]))
plt.xlabel('Day')
plt.ylabel('Infectious')
plt.gcf().autofmt_xdate()
plt.grid()
plt.legend()
plt.show()

In [21]:
observe_cities = list(origin_data['Country'])[:6]

In [ ]:
plt.figure(figsize=(12, 9))
plt.subplots_adjust(wspace=0.2, hspace=0.4)
for i in range(len(observe_cities)):
    plt.subplot(3, 2, i+1)
    
    cityid = city_ID[observe_cities[i]]
    
    plt.plot(alldates_dt, epidemic[cityid, 2:], 'k+-', label = 'truth')
    plt.plot(t_range_dt, I[cityid, :] + R[cityid, :], 'r--', label = 'predict')
    plt.title(u'{} (Start from {})'.format(observe_cities[i], alldates[0]))
    plt.xlabel('Day')
    plt.ylabel('Infectious')
    plt.gcf().autofmt_xdate()
    plt.grid()
    plt.legend()
plt.show()

In [23]:
observe_cities = list(origin_data['Country'])[6:12]

In [ ]:
plt.figure(figsize=(12, 9))
plt.subplots_adjust(wspace=0.2, hspace=0.4)
for i in range(len(observe_cities)):
    plt.subplot(3, 2, i+1)
    
    cityid = city_ID[observe_cities[i]]
    
    plt.plot(alldates_dt, epidemic[cityid, 2:], 'k+-', label = 'truth')
    plt.plot(t_range_dt, I[cityid, :] + R[cityid, :], 'r--', label = 'predict')
    plt.title(u'{} (Start from {})'.format(observe_cities[i], alldates[0]))
    plt.xlabel('Day')
    plt.ylabel('Infectious')
    plt.gcf().autofmt_xdate()
    plt.grid()
    plt.legend()
plt.show()